# VLM Benchmark for Object Property Abstraction

This notebook implements a benchmark for evaluating Vision Language Models (VLMs) on object property abstraction and visual question answering (VQA) tasks. The benchmark includes three types of questions:

1. Direct Recognition
2. Property Inference
3. Counterfactual Reasoning

And three types of images:
- REAL
- ANIMATED
- AI GENERATED

## Setup and Imports

First, let's import the necessary libraries and set up our environment.

In [ ]:
# Install required packages
# %pip install transformers torch Pillow tqdm bitsandbytes accelerate

In [ ]:
%pip install qwen-vl-utils flash-attn #--no-build-isolation

In [4]:
# Import required libraries
import torch
import json
from pathlib import Path
from PIL import Image
import gc
import re
from tqdm import tqdm
from typing import List, Dict, Any
from qwen_vl_utils import process_vision_info

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


## Benchmark Tester Class

This class handles the evaluation of models against our benchmark.

In [8]:
class BenchmarkTester:
    def __init__(self, benchmark_path="/var/scratch/ave303/OP_bench/benchmark.json", data_dir="/var/scratch/ave303/OP_bench/"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        with open(benchmark_path, 'r') as f:
            self.benchmark = json.load(f)
        self.data_dir = data_dir
    
    def format_question(self, question, model_name):
        """Format a question for the model."""

        if model_name=="blip2":
            return f"Question: {question['question']} Answer:"
        else:
            return f"Question: {question['question']} Answer with a number and list of objects. Answer:"

    def clean_answer(self, answer):
        """Clean the model output to extract just the number."""
        # Remove any text that's not a number
        # import re
        # numbers = re.findall(r'\d+', answer)
        # if numbers:
        #     return numbers[0]  # Return the first number found
        # return answer
        """Extract number and reasoning from the model's answer."""
        # Try to extract number and reasoning using regex
        import re
        pattern = r'(\d+)\s*\[(.*?)\]'
        match = re.search(pattern, answer)
        
        if match:
            number = match.group(1)
            objects = [obj.strip() for obj in match.group(2).split(',')]
            return {
                "count": number,
                "reasoning": objects
            }
        else:
            # Fallback if format isn't matched
            numbers = re.findall(r'\d+', answer)
            return {
                "count": numbers[0] if numbers else "0",
                "reasoning": []
            }

    def model_generation(self, model_name, model, inputs, processor):
        """Generate answer and decode."""
        outputs = None  # Initialize outputs to None
        
        if model_name=="smolVLM2":
            outputs = model.generate(**inputs, do_sample=False, max_new_tokens=64)
            answer = processor.batch_decode(
                outputs,
                skip_special_tokens=True,
            )[0]
        elif model_name=="Qwen2.5-VL":
            outputs = model.generate(**inputs, max_new_tokens=128)
            outputs = [
                out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, outputs)
            ]
            answer = processor.batch_decode(
                outputs, skip_special_tokens=True, clean_up_tokenization_spaces=False
            )[0]
        else:
            print(f"Warning: Unknown model name '{model_name}' in model_generation.")
            answer = ""  # Return an empty string

        return answer, outputs
    
    def evaluate_model(self, model_name, model, processor, save_path, start_idx=0, batch_size=5):
        results = []
        print(f"\nEvaluating {model_name}...")
        print(f"Using device: {self.device}")
        
        # Force garbage collection before starting
        gc.collect()
        torch.cuda.empty_cache()

        try:
            images = self.benchmark['benchmark']['images'][start_idx:start_idx + batch_size]
            total_images = len(images)
            
            for idx, image_data in enumerate(tqdm(images, desc="Processing images")):
                try:
                    print(f"\nProcessing image {idx+1}/{total_images}: {image_data['image_id']}")
                    image_path = Path(self.data_dir)/image_data['path']
                    if not image_path.exists():
                        print(f"Warning: Image not found at {image_path}")
                        continue
                    
                    # Load and preprocess image
                    image = Image.open(image_path).convert("RGB")
                    image_results = []  # Store results for current image
                    
                    for question in image_data['questions']:
                        try:
                            # prompt = self.format_question(question, model_name)
                            print(f"Question: {question['question']}")

                            messages = [
                                {
                                    "role": "user",
                                    "content": [
                                        {"type": "image", "image": image},
                                        # {"type": "text", "text": f"{question['question']} Answer format: total number(numerical) objects(within square brackets)"},
                                        # {"type": "text", "text": f"{question['question']} Provide just the total count and the list of objects in the given format \n Format: number [objects]"},
                                        {"type": "text", "text": f"{question['question']} Answer Format: number [objects]"}
                                    ]
                                },
                            ]
                            
                            # Clear cache before processing each question
                            torch.cuda.empty_cache()
                            
                            # Process image and text
                            # inputs = processor(images=image, text=prompt, return_tensors="pt").to(self.device)
                            if model_name=="smolVLM2":
                                inputs = processor.apply_chat_template(
                                    messages,
                                    add_generation_prompt=True,
                                    tokenize=True,
                                    return_dict=True,
                                    return_tensors="pt",
                                ).to(model.device, dtype=torch.float16)
                            else:
                                
                                text = processor.apply_chat_template(
                                    messages, tokenize=False, add_generation_prompt=True
                                )
                                # image_inputs, video_inputs = process_vision_info(messages)
                                inputs = processor(
                                    text=text,
                                    images=image,
                                    videos=None,
                                    padding=True,
                                    return_tensors="pt",
                                ).to("cuda")
                            
                            # Generate answer with better settings
                            with torch.no_grad():
                                answer, outputs = self.model_generation(model_name, model, inputs, processor)    #call for model.generate
        
                            cleaned_answer = self.clean_answer(answer)
                            
                            image_results.append({
                                "image_id": image_data["image_id"],
                                "image_type": image_data["image_type"],
                                "question_id": question["id"],
                                "question": question["question"],
                                "ground_truth": question["answer"],
                                "model_answer": cleaned_answer["count"],
                                "model_reasoning": cleaned_answer["reasoning"],
                                "raw_answer": answer,  # Keep raw answer for debugging
                                "property_category": question["property_category"]
                            })
                            
                            # Clear memory
                            del outputs, inputs
                            torch.cuda.empty_cache()
                            
                        except Exception as e:
                            print(f"Error processing question: {str(e)}")
                            continue
                    
                    # Add results from this image
                    results.extend(image_results)
                    
                    # Save intermediate results only every 2 images or if it's the last image
                    if (idx + 1) % 2 == 0 or idx == total_images - 1:
                        with open(f"{save_path}_checkpoint.json", 'w') as f:
                            json.dump(results, f, indent=4)
                            
                except Exception as e:
                    print(f"Error processing image {image_data['image_id']}: {str(e)}")
                    continue
            
            # Save final results
            if results:
                with open(save_path, 'w') as f:
                    json.dump(results, f, indent=4)
            
        except Exception as e:
            print(f"An error occurred during evaluation: {str(e)}")
            if results:
                with open(f"{save_path}_error_state.json", 'w') as f:
                    json.dump(results, f, indent=4)
        
        return results

## Test SmolVLM Model

Let's evaluate the SmolVLM2-2.2B-Instruct model

In [10]:
# def test_smolVLM2():
#     from transformers import AutoProcessor, AutoModelForImageTextToText

#     print("Loading smolVLM model...")
    
#     model = AutoModelForImageTextToText.from_pretrained(
#         "HuggingFaceTB/SmolVLM2-2.2B-Instruct",
#         torch_dtype=torch.float16,
#         attn_implementation="flash_attention_2",
#         low_cpu_mem_usage=True,
#         trust_remote_code=True
#     ).to("cuda")

#     processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM2-2.2B-Instruct")

#     ## A bit slow without the flash_attention2 requires ampere gpu's. Better performance in some cases

#     # Optional: Enable memory efficient attention
#     if hasattr(model.config, 'use_memory_efficient_attention'):
#         model.config.use_memory_efficient_attention = True

#     tester = BenchmarkTester()
#     smolVLM_results = tester.evaluate_model(
#         "smolVLM2",
#         model, 
#         processor, 
#         "smolVLM2_results_1.json", 
#         batch_size=25
#     )

#     # Clean up
#     del model, processor
#     torch.cuda.empty_cache()
#     gc.collect()

## Test Qwen2.5-VL

Lets evaluate the Qwen2.5-VL-7B-Instruct model

In [10]:
def test_Qwen2_5VL():
    from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
    
    # default: Load the model on the available device(s)
    # model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    #     "Qwen/Qwen2.5-VL-3B-Instruct", 
    #     load_in_8bit=True, # throws error when .to() is added
    #     torch_dtype=torch.bfloat16, 
    #     device_map="auto",
    #     # attn_implementation="flash_attention_2",
    #     low_cpu_mem_usage=True
    # )
    
    # We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
    model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
        "/var/scratch/ave303/models/qwen2-5-vl-32b",
        torch_dtype=torch.float16,
        attn_implementation="flash_attention_2",
        device_map="auto",
        low_cpu_mem_usage=True,
        trust_remote_code=True
    )
    
    # default processer
    processor = AutoProcessor.from_pretrained("/var/scratch/ave303/models/qwen2-5-vl-32b")

    ### Qwen2.5-VL-7B-Instruct --> goes out of CUDA memory
    ### Qwen2.5-VL-3B-Instruct --> can handle only 2 images before going out of memory but decent performance

    # Optional: Enable memory efficient attention
    if hasattr(model.config, 'use_memory_efficient_attention'):
        model.config.use_memory_efficient_attention = True

    tester = BenchmarkTester()
    Qwen2_5VL_results = tester.evaluate_model(
        "Qwen2.5-VL",
        model, 
        processor, 
        "Qwen2.5-VL_32b_results.json", 
        batch_size=50
    )

    # Clean up
    del model, processor
    torch.cuda.empty_cache()
    gc.collect()

## Run Evaluation

Now we can run our evaluation. Let's start with the SmolVLM2 model:

In [ ]:
# test_smolVLM2()

In [ ]:
test_Qwen2_5VL()